In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 표를 나타내기 위해 seaborn과 matplotlib를 부르고
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# 표에 한글을 표시하기 위해 폰트를 맑은(malgun.ttf)로 설정합니다.
import matplotlib.font_manager as fm
font = fm.FontProperties(fname='../input/korea-corona-case-to-211227/malgun.ttf')
font_name = fm.FontProperties(fname='../input/korea-corona-case-to-211227/malgun.ttf').get_name()
mpl.rc('font',family=font_name)

# 경고 메시지 나오지 않게
#import warnings
#warnings.filterwarnings("ignore")

## 1. 이 노트북을 통해 배울 수 있는 내용
- 1) 막대 그래프, 원형 그래프 표현(bar, pie, countplot 등)
- 2) 판다스에서 원하는 자료 불러오기(loc, iloc)
- 3) 특정 열 기준으로 자료 정렬(groupby)
- 4) 람다
- 5) 세월호에 누가 얼마나 탔는지, 누가 못 돌아왔는지

In [ ]:
# 세월호 탑승자 명단을 부릅니다.
세월호 = pd.read_csv('../input/korea-corona-case-to-211227/sewol_etd.csv')
세월호.head(2)

In [ ]:
# 이 표 세월호 탑승자 명단은 인터넷 기사와, 자료를 수집, 정리해 만들었으며 총 476행(row) 3열(column)로 이루어져 있습니다. 

print(세월호.shape)
print(세월호.info())

In [ ]:
# '세월호'를 복사해 다시 '세월'로 저장
세월 = 세월호.copy()
세월

## 2. 열 관련 정보
    탑승자를 '대분류'에서 '선원', '단원고', '일반'으로 구분했고, '중분류'에서는 '선원'의 경우 배를 조종하는 데 관련이 있는 선원은 '선박직'으로 승객 대응에 관련이 있는 선원은 '비선박직'으로 구분했고, '단원고'의 경우 '교사'와 '학생'으로 구분했고, '일반'은 구분하지 않았습니다. '생사'의 경우 생존했을 경우 '생', 사망했을 경우 '사'로 구분했습니다.
    
|번호|열(column)|종류|개수|정보|내용|
| --- | --- | --- | --- | --- | --- |
|1|대분류|범주형|3|탑승자 구분|선원, 단원고, 일반|
|2|중분류|범주형|5|선원은 선박직, 비선박직으로, 단원고는 학생, 교사로, 일반 일반으로|선박직, 비선박직, 학생, 교사, 일반|
|3|생사|범주형|2|사고 직후 생사 여부|생존, 사망|

In [ ]:
# 본격적으로 각 열을 분석하기 전에 표 전체에 대해 좀 더 알아보겠습니다.
세월.columns

In [ ]:
# 포문(for)을 써서 열마다 각각 몇 개의 값이 있는지, 그 값들이 무엇인지 확인
# 프린트에 나오는 값은 순서대로 열 제목, 열마다 몇 개의 값으로 이루어졌는지, 값이 무엇무엇인지입니다.
for 열 in 세월.columns:
    print(열, len(세월[열].unique()),'개', 세월[열].unique()) 

In [ ]:
세월.describe()

In [ ]:
# 각 열별 범주형 변수 갯수 확인
for 열 in 세월.columns:
    print(세월[열].value_counts(), '\n')

## 3. 각 '열' 분석 및 시각화
## 3.1 '대분류'열 분석 및 시각화

In [ ]:
# 우선 첫 번째 '대분류' 열을 살펴 보겠습니다.
세월['대분류']

In [ ]:
# 다행히 결측치가 없습니다.
세월['대분류'].isnull().sum()

In [ ]:
len(세월['대분류'])

In [ ]:
# '대분류' 열에 범주형 변수가 각각 몇 개가 있는지 알기 위해 value_counts()를 사용합니다.
# 전체 인원 476명 중 '단원고'의 숫자가 339명으로 제일 많습니다.
세월['대분류'].value_counts()

In [ ]:
# 바로 위 결과를 데이터프레임(표)으로 만들고 '세월_대'로 정의
세월_대 = pd.DataFrame(세월['대분류'].value_counts()) 
세월_대

### 3.1.1 람다(lambda) 활용해 '단원고', '일반', '세월호 선원' 비율 나타내기
    - 람다(lambda)와 포(for), 같은 결과를 얻을 수 있지만 방법이 다릅니다.

In [ ]:
# 람다 함수를 통해 전체 탑승 인원 수(476이 아닌, 세월_대.sum() 활용)를 '단원고', '일반', '세월호 선원' 탑승 인원 수로 나눈 후, 
# 라운드를 통해 소수점 2째자리까지 반올림하고, 백분율 계산을 위해 100을 곱합니다.
# '세월_대' 데이터프레임에 새로운 열 '대분류_비율 추가'

세월_대['대분류_비율'] = round(세월_대['대분류'].apply(lambda x : x / 세월_대.sum()), 2) * 100
세월_대

### 3.1.2 프린트(print)와 포맷(format) 활용해 수치 나타내기

In [ ]:
# 위 내용을 정리하면 아래처럼 나타낼 수 있습니다. 
print('세월호 총 탑승 인원은 476명으로 단원고 339명, 일반 104명, 선원 33명이 탔다.')
print('단원고 탑승자 비율이 71%, 일반이 22%, 세월호 선원이 7%였다.')

In [ ]:
# 같은 내용을 포맷을 써서 정리해 보면
print('세월호 총 탑승 인원은 {}명이다.'.format(세월_대['대분류'].sum()))

print('단원고 {}명, 일반 {}명, 선원 {}명이 탔다'
      .format(세월_대.loc['단원고','대분류'], 세월_대.loc['일반','대분류'], 세월_대.loc['선원','대분류']))

print('단원고 탑승자 비율이 {:.0f}%, 일반이 {:.0f}%, 세월호 선원이 {:.0f}%였다.'
      .format(세월_대.loc['단원고','대분류_비율'], 세월_대.loc['일반','대분류_비율'], 세월_대.loc['선원','대분류_비율']))

### 3.1.3 데이터프레임 값 갖고 오기(loc, iloc 활용)
 - loc, iloc은 데이터 프레임 값을 갖고 올 때 자주 씁니다. 차이는
  
     - 1) iloc은 숫자로 불러와야 한다면, loc은 숫자와 문자를 섞어서 불러올 수 있습니다.
     - 2) iloc은 마지막 값을 포함시키지 않지만, loc은 마지막 값을 포함합니다.
     
- 정도입니다. 위에서 만든 데이터 프레임 '세월_대'에서 값 339를 꺼내려면 

In [ ]:
세월_대

In [ ]:
# 값 339의 위치가 행(row) 기준으로 가장 첫 번째(0), 열(column) 기준으로도 가장 첫 번째(0)입니다.
세월_대.iloc[0, 0]

In [ ]:
# 값 339의 위치가 '단원고' 행에서 '대분류' 열에 있습니다.
세월_대.loc['단원고', '대분류']

In [ ]:
# iloc은 마지막 값(0:3에서 3)을 제외한 행 기준 첫 번째(0), 두 번째(1), 세 번째(2) 값을 불러 옵니다.
세월_대.iloc[0:3, 0]

In [ ]:
# loc은 마지막 값('단원고':'세월호 선원'에서 '세월호 선원')을 포함시켜 불러 옵니다.
세월_대.loc['단원고':'선원', '대분류']

### 3.1.4 맷플롯을 활용해 막대와 원 그래프로 시각화

In [ ]:
# 막대 그래프
ax = 세월_대['대분류'].plot.bar()
ax.set_xticklabels(세월['대분류'].value_counts().index, fontproperties=font, rotation = 0)

In [ ]:
# 원형 그래프
plt.pie(세월_대['대분류_비율']);
plt.legend(세월_대['대분류_비율'].index, prop=font)
plt.show()

In [ ]:
# 이 막대와 원 그래프를 한 줄에 표시합니다.

plt.figure(figsize=(12,6)) # 전체 크기 설정 가로 12인치(30.48센티미터), 세로 6인치(15.24센티미터)

plt.subplot(1,2,1) # 1행 2열의 그래프 중 1번 째
ax = 세월_대['대분류'].plot.bar()
ax.set_xticklabels(세월['대분류'].value_counts().index, fontproperties=font, rotation = 0)

plt.subplot(1,2,2) # 1행 2열의 그래프 중 2번 째
plt.pie(세월_대['대분류_비율']);
plt.legend(세월_대['대분류_비율'].index, prop=font)

plt.show()

In [ ]:
# 보기 좋게 색깔도 넣고, 글자도 가로로 보기 편하게 하고, 그래프 제목도 넣습니다.
plt.figure(figsize=(12,6))

plt.subplot(1,2,1)
plt.title('세월호 탑승자', fontproperties=font)
# 각각을 막대 그래프로 만들고 (단원고 : yellow, 일반 : greenyellow, 세월호 선원 : gray)
ax = 세월_대['대분류'].plot.bar(color = ['yellow', 'greenyellow', 'gray'])
ax.set_xticklabels(세월['대분류'].value_counts().index, fontproperties=font, rotation = 0)

plt.ylabel('명', fontproperties=font, rotation = 0) # y축 수치가 인원수라는 걸 알리기 위해 '명'을 쓰고, 기본값이 90도로 기울어져 있기에 기울기를 0으로 조정
plt.xlabel('대분류', fontproperties=font) # x축이 '대분류' 열이라는 걸 표시

for i, v in enumerate(세월_대['대분류']): # i는 순서를 나타내는 index, v는 값을 나타내는 value
    plt.text(x = i, y = v , s = f"{v}", color='black', ha = 'center', va= 'bottom') 
    # 막대 그래프 위에 나오는 숫자를 표시하는 방법, x,y는 숫자를 표시할 위치, s는 표시할 값, color는 글자색, ha, va는 세부 위치

plt.subplot(1,2,2)
plt.title('세월호 탑승자 비율', fontproperties=font)
plt.pie(세월_대['대분류'], 
        # labels = 세월_대['대분류'].index, 
        explode = (0.1, 0.1, 0.1), # 부채꼴 끼리의 간격
        colors = ['yellow', 'greenyellow', 'gray'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%') # 수치 나타냄
plt.legend(세월_대['대분류_비율'].index, prop=font)

plt.show()

In [ ]:
# 시본(sns.countplot)을 활용해서도 마찬가지 막대그래프로 나타낼 수 있습니다.
ax = sns.countplot(x = '대분류', data = 세월);
ax.set_xticklabels(세월['대분류'].unique(), fontproperties=font, rotation = 0)
plt.xlabel('대분류', fontproperties=font)
plt.show()

## 3.2 '대분류', '생사'열 분석 및 시각화

In [ ]:
# 생존자와 사망자 비중
ax = sns.countplot(x = '생사', data = 세월);
ax.set_xticklabels(세월['생사'].unique(), fontproperties=font, rotation = 0)
plt.xlabel('대분류', fontproperties=font)
plt.show()

In [ ]:
세월_생사 = pd.DataFrame(세월['생사'].value_counts())
세월_생사

In [ ]:
세월_생사['생사_비율'] = round(세월_생사['생사'].apply(lambda x : x / 세월_생사['생사'].sum()),2) * 100
세월_생사

- 사망자의 비율이 64%임을 알 수 있습니다.

In [ ]:
# '대분류' 열 기준으로 '생사(사건 직후)'열 값을 groupby로 묶은 후 개수를(value_counts()) 데이터프레임으로 나타냅니다.
대_생사 = pd.DataFrame(세월.groupby(['대분류'])['생사'].value_counts())
대_생사

In [ ]:
대_생사['비율'] = round(대_생사['생사'].apply(lambda x : x / 대_생사['생사'].sum()),2)*100
대_생사

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(1,2,1)
plt.title('생사 비율', fontproperties=font, fontsize = 15)
ax = 대_생사['생사'].plot.bar(color = ['yellow','beige', 'silver', 'gray', 'greenyellow', 'forestgreen']) 
ax.set_xticklabels(대_생사['생사'].index, fontproperties=font, rotation = 80)
# 각각을 막대 그래프로 만들고 (단원고 : yellow, 일반 : greenyellow, 세월호 선원 : gray)

plt.ylabel('명', fontproperties=font, rotation = 0) # y축 수치가 인원수라는 걸 알리기 위해 '명'을 쓰고, 기본값이 90도로 기울어져 있기에 기울기를 0으로 조정
plt.xlabel('대분류', fontproperties=font) # x축이 '대분류' 열이라는 걸 표시

for i, v in enumerate(대_생사['생사']): # i는 순서를 나타내는 index, v는 값을 나타내는 value
    plt.text(x = i, y = v , s = f"{v}", color='black', ha = 'center', va= 'bottom') 
    # 막대 그래프 위에 나오는 숫자를 표시하는 방법, x,y는 숫자를 표시할 위치, s는 표시할 값, color는 글자색, ha, va는 세부 위치

plt.subplot(1,2,2)
plt.title('생사 비율', fontproperties=font, fontsize = 15)
plt.pie(대_생사['생사'], 
        # labels = 대_생사['생사'].index, 
        explode = (0.1,0.1, 0.1,0.1, 0.1,0.1), # 부채꼴끼리 간격
        colors = ['yellow','beige', 'silver', 'gray', 'greenyellow', 'forestgreen'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%',
        startangle = 80) # 수치 나타냄
plt.legend(대_생사['생사'].index, prop=font)

plt.show()

plt.tight_layout() #그래프 간 간격 조정

print('총 탑승자 {}명 중'.format(대_생사['생사'].sum()))

print('단원고 사망자 {:.0f}명, 생존자 {:.0f}명'
      .format(대_생사.loc[('단원고', '사망'), '생사'], 대_생사.loc[('단원고', '생존'), '생사']))

print('일반 사망자 {}명, 생존자 {}명'.format(대_생사.iloc[5,0], 대_생사.iloc[4,0])) 

print('세월호 선원 사망자 {:.0f}명, 생존자 {:.0f}명'
      .format(대_생사.loc[('선원', '사망'), '생사'], 대_생사.loc[('선원', '생존'), '생사']))

print('단원고 사망자가 차지하는 비중은 {:.0f}%, 일반 사망자 비중은 {:.0f}%, 세월호 선원 사망자 비중은 {:.0f}%'
      .format(대_생사.iloc[0,1], 대_생사.iloc[5,1], 대_생사.iloc[3,1]))

- 세월호 선원과 일반 모두 생존자가 사망자보다 많으나, 단원고만 사망자가 생존자보다 많습니다.

In [ ]:
ax = sns.countplot(hue = '생사', x = '대분류', data = 세월);
ax.set_xticklabels(세월['대분류'].unique(), fontproperties=font, rotation = 0)
plt.xlabel('대분류', fontproperties=font)
plt.legend(세월['생사'].unique(), prop=font)
plt.show()

- 각 집단에서 생사 비중이 어떻게 되는지 살펴보겠습니다.

In [ ]:
대_생사

In [ ]:
각_비율 = []
for i, j in zip(대_생사['생사'], 대_생사.index): # zip( , )으로 묶어 주면 한 번에 두 개의 값을 불러올 수 있다.
    # print(round(i / 대_생사.loc[j[0], '생사'].sum(),2))
    각_비율.append(round(i / 대_생사.loc[j[0], '생사'].sum(),2) * 100)
대_생사['각_비율'] = 각_비율

In [ ]:
대_생사

In [ ]:
plt.figure(figsize=(18,7))

plt.subplot(1,3,1)
plt.title('단원고 집단의 생사 비중', fontproperties=font, fontsize = 15)
plt.pie(대_생사.loc[('단원고'), '생사'], 
        explode = (0.1,0.1), # 원 조각끼리 간격
        colors = ['red','yellow'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%',
        startangle = 70) # 기울기기울기
plt.legend(대_생사.loc[('단원고'), '생사'].index, prop=font)

plt.subplot(1,3,2)
plt.title('일반 집단의 생사 비중', fontproperties=font, fontsize = 15)
plt.pie(대_생사.loc[('일반'), '생사'], 
        explode = (0.1,0.1), # 원 조각끼리 간격
        colors = ['limegreen','red'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%',
        startangle = 250) # 기울기
plt.legend(대_생사.loc[('일반'), '생사'].index, prop=font)


plt.subplot(1,3,3)
plt.title('세월호 선원 집단의 생사 비중', fontproperties=font, fontsize = 15)
plt.pie(대_생사.loc[('선원'), '생사'], 
        explode = (0.1,0.1), # 부채꼴끼리 간격
        colors = ['gray','red'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%',
        startangle = 250) # 기울기
plt.legend(대_생사.loc[('선원'), '생사'].index, prop=font)

plt.show()

print('단원고 {}명 중 사망자는 {:.0f}명이고 비율은 {:.0f}%다.'
.format(
    대_생사.loc[('단원고'), '생사'].sum(), 
    대_생사.loc[('단원고', '사망'), '생사'],
    대_생사.loc[('단원고', '사망'), '각_비율'])
     )

print('일반 {}명 중 사망자는 {:.0f}명이고 비율은 {:.0f}%다.'
.format(
    대_생사.loc[('일반'), '생사'].sum(), 
    대_생사.loc[('일반', '사망'), '생사'],
    대_생사.loc[('일반', '사망'), '각_비율'])
     )

print('세월호 선원 {}명 중 사망자는 {:.0f}명이고 비율은 {:.0f}%다.'
.format(
    대_생사.loc[('선원'), '생사'].sum(), 
    대_생사.loc[('선원', '사망'), '생사'],
    대_생사.loc[('선원', '사망'), '각_비율'])
     )

## 3.3 '대분류', '중분류', '생사'열 분석 및 시각화

In [ ]:
# 대분류, 중분류를 활용해 사망 여부 파악
대_중_생사 = pd.DataFrame(세월.groupby(['대분류', '중분류'])['생사'].value_counts())
대_중_생사

In [ ]:
# 총합이 476이 아니라 449인 이유는 '중분류'열의 결측치 때문
대_중_생사.sum()

In [ ]:
# '중분류' 열의 결측치는 27개, 위의 449개와 더하면 476, 총 인원 수와 같다.
세월['중분류'].isna().sum()

In [ ]:
대_중_생사['비율'] = round(대_중_생사['생사'].apply(lambda x : x / 대_중_생사['생사'].sum()),2)*100
대_중_생사

- 가장 큰 수치는 250으로 단원고 학생 사망자 수입니다.
- 중분류 집단별 생존자와 사망자의 비중을 살펴보겠습니다.

In [ ]:
각_비율_2 = []
for i, j in zip(대_중_생사['생사'], 대_중_생사.index): # zip( , )으로 묶어 주면 한 번에 두 개의 값을 불러올 수 있습니다.
    #print(round(i / 대_중_생사.loc[j[:2], '생사'].sum(),2))
    각_비율_2.append(round(i / 대_중_생사.loc[j[:2], '생사'].sum(),2) * 100)

대_중_생사['각_비율'] = 각_비율_2

In [ ]:
대_중_생사

In [ ]:
plt.figure(figsize=(15,15))

plt.subplot(2,3,1)
plt.title('단원고 학생의 생사 비중\n사망 250명, 생존 75명', fontproperties=font, fontsize = 15)
plt.pie(대_중_생사.loc[('단원고', '학생'), '생사'], 
        explode = (0.1,0.1), # 원 조각끼리 간격
        colors = ['red','yellow'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%',
        startangle = 70) # 기울기
plt.legend(대_중_생사.loc[('단원고', '학생'), '생사'].index, prop=font)

plt.subplot(2,3,2)
plt.title('단원고 교사의 생사 비중\n사망 11명, 생존 3명', fontproperties=font, fontsize = 15)
plt.pie(대_중_생사.loc[('단원고', '교사'), '생사'],  
        explode = (0.1,0.1), # 원 조각끼리 간격
        colors = ['red','yellow'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%',
        startangle = 70) # 기울기
plt.legend(대_중_생사.loc[('단원고', '교사'), '생사'].index, prop=font)

plt.subplot(2,3,3)
plt.title('세월호 선박직의 생사 비중\n사망 0명 생존 15명', fontproperties=font, fontsize = 15)
plt.pie(대_중_생사.loc[('선원', '선박직'), '생사'], 
        colors = ['gray'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%',
        startangle = 70) # 기울기기울기
        #생존율이 100%니 부채꼴 사이의 간격을 조정하는 explode가 필요 없다.
plt.legend(대_중_생사.loc[('선원', '선박직'), '생사'].index, prop=font)

plt.subplot(2,3,4)
plt.title('세월호 비선박직의 생사 비중\n사망 10명, 생존 8명', fontproperties=font, fontsize = 15)
plt.pie(대_중_생사.loc[('선원', '비선박직'), '생사'],
        explode = (0.1,0.1),
        colors = ['red', 'gray'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%',
        startangle = 70) # 기울기기울기
        #생존율이 100%니 부채꼴 사이의 간격을 조정하는 explode가 필요 없다.
plt.legend(대_중_생사.loc[('선원', '비선박직'), '생사'].index, prop=font)

plt.subplot(2,3,5)
plt.title('일반 목적으로\n탄 사람의 생사 비중\n사망 33명, 생존 71명', fontproperties=font, fontsize = 15)
plt.pie(대_중_생사.loc[('일반', '일반'), '생사'], 
        explode = (0.1,0.1), # 원 조각끼리 간격
        colors = ['limegreen','red'], # 색깔(막대 그래프 색깔(color) 설정과 다르게 colors, 복수다.)
        autopct = '%.0f%%',
        startangle = 240)
plt.legend(대_중_생사.loc[('일반', '일반'), '생사'].index, prop=font)

plt.show()
plt.tight_layout()



plt.figure(figsize=(8,6))
plt.title('각 집단별 생존자, 사망자', fontproperties=font, fontsize = 15)
ax = 대_중_생사['생사'].plot.bar(color = ['red','yellow', 'red', 'yellow', 'red', 'gray', 'gray', 'limegreen', 'red'])
ax.set_xticklabels(대_중_생사['생사'].index, fontproperties=font, rotation = 90)
plt.ylabel('명', fontproperties=font, rotation = 0)
plt.xlabel('대, 중 분류에 따른 구분', fontproperties=font)

for i, v in enumerate(대_중_생사['생사']): # i는 순서를 나타내는 index, v는 값을 나타내는 value
    plt.text(x = i, y = v , s = f"{v}", color='black', ha = 'center', va= 'bottom')
plt.show()

## 4. 마무리
 - 이 노트북에서는 세월호 탑승자 명단 기본 정보를 알아보고, '대분류', '중분류'에 따라 생사 여부를 구분해보았습니다.